# Auto Insurance Claim Frequency Modeling

**Author:** Ben Honil 
**Institution:** ESILV - École Supérieure d'Ingénieurs Léonard de Vinci  
**Program:** Master 1 - Actuarial Science  
**Date:** December 2024

## 1. Data Loading and Libraries

In [1]:
import pandas as pd
import numpy as np

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and Models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression, PoissonRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_poisson_deviance

### 1.1 Loading the Dataset

We use the `freMTPL2freq` dataset (French Motor Third-Party Liability claim frequency).

In [2]:
df = pd.read_csv("freMTPL2freq.csv")
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDpol       678013 non-null  float64
 1   ClaimNb     678013 non-null  int64  
 2   Exposure    678013 non-null  float64
 3   Area        678013 non-null  object 
 4   VehPower    678013 non-null  int64  
 5   VehAge      678013 non-null  int64  
 6   DrivAge     678013 non-null  int64  
 7   BonusMalus  678013 non-null  int64  
 8   VehBrand    678013 non-null  object 
 9   VehGas      678013 non-null  object 
 10  Density     678013 non-null  int64  
 11  Region      678013 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 62.1+ MB


### 1.2 General Description

Each row corresponds to an auto insurance contract.

**Main variables:**
- `IDpol`: Policy identifier (technical, not predictive)
- `ClaimNb`: Number of claims observed during the period (target variable)
- `Exposure`: Exposure duration (in years, between 0 and 1)
- `Area`, `Region`: Geographic information
- `VehPower`, `VehAge`, `VehBrand`, `VehGas`: Vehicle characteristics
- `DrivAge`: Driver's age
- `BonusMalus`: Bonus-malus coefficient
- `Density`: Population density of the residence area

In [3]:
df.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


## 2. Descriptive Analysis and Visualization

We first examine the distribution of the target variable `ClaimNb` and the exposure `Exposure`. We also create the frequency variable.

In [4]:
# Create frequency: ClaimNb / Exposure
df['Frequency'] = df['ClaimNb'] / df['Exposure']

df.describe()

,IDpol,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density,Frequency
count,678013,678013,678013,678013,678013,678013,678013,678013,678013
mean,3.44e+06,0.053,0.528,6.48,7.04,45.5,59.8,1792,0.197
std,1.96e+06,0.238,0.318,2.14,5.67,14.2,15.6,3958,1.84
min,1.0,0,0.002,4,0,18,50,1,0
max,6.11e+06,4,1.0,15,100,100,230,27000,500


In [5]:
# Distribution of ClaimNb
print("ClaimNb Distribution:")
for i in range(5):
    count = (df['ClaimNb'] == i).sum()
    pct = count / len(df) * 100
    print(f"{i} claims: {count:,} ({pct:.2f}%)")

print(f"\nZero-inflation rate: {(df['ClaimNb'] == 0).sum() / len(df) * 100:.2f}%")

ClaimNb Distribution:
0 claims: 643,212 (94.87%)
1 claims: 32,986 (4.87%)
2 claims: 1,715 (0.25%)
3 claims: 93 (0.01%)
4 claims: 7 (0.00%)

Zero-inflation rate: 94.87%


In [6]:
# Correlation between numerical variables
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", annot=True)
plt.title("Correlation Matrix (Numerical Variables)")
plt.show()

**Interpretation:**

The correlation matrix reveals only weak correlations between numerical variables and the target variable.
This is expected in a context of highly imbalanced claim data (ClaimNb ≈ 0 for the majority of contracts).
Linear correlation is not suitable for evaluating variable relevance in a Poisson or non-linear model.
Variables may have real predictive importance despite near-zero correlation.

**Key Observations:**

- The mean of `ClaimNb` is very low (~0.05), meaning a large majority of contracts have no claims (highly imbalanced data)
- The average frequency (`ClaimNb / Exposure`) is also low, with many zeros and some very high values when a claim occurs on short exposure
- This behavior is typical of **rare count data**, well-suited for Poisson regression

## 3. Data Cleaning and Preprocessing

We apply simple cleaning rules to remove outliers:
- Remove drivers under 18 years old
- Keep vehicles with reasonable age (≤ 30 years)
- Remove rows with zero Exposure (to avoid division by zero)

In [7]:
# Simple outlier filtering
df = df[df['DrivAge'] >= 18]
df = df[df['VehAge'] <= 30]
df = df[df['Exposure'] > 0]

df.shape

(676897, 13)

### 3.1 Categorical Variable Encoding

Scikit-learn models don't handle categorical variables directly, so we use **one-hot encoding** via `pd.get_dummies`, dropping one reference category (`drop_first=True`).

In [8]:
cat_cols = ['Area', 'VehBrand', 'VehGas', 'Region']

df1 = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df1.shape

(676897, 46)

The `drop_first=True` option removes the first category of each categorical variable, avoiding **perfect collinearity** (the missing category can be inferred from the others).

## 4. Target Definition and Train/Test Split

We aim to predict the **number of claims `ClaimNb`** from other variables.

We remove from X:
- `ClaimNb` (target)
- `Frequency` (derived from ClaimNb and Exposure)
- `IDpol` (technical identifier, not informative)

We keep `Exposure` in df1 to potentially use it as **weights** in Poisson regression.

In [9]:
# Target variable
y = df1['ClaimNb']

# Features: remove ClaimNb, Frequency and IDpol
X = df1.drop(columns=['ClaimNb', 'Frequency', 'IDpol'])

# Keep Exposure separately for Poisson regression (weights)
exposure = df1['Exposure']

X_train, X_test, y_train, y_test, exp_train, exp_test = train_test_split(
    X, y, exposure, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

((541517, 43), (135380, 43))

## 5. Model Training

We compare several regression models on the count variable `ClaimNb`:

- Linear Regression (baseline, not theoretically suitable but useful as simple reference)
- Decision Tree
- Random Forest
- Gradient Boosting
- Linear SVR (maximum margin model)
- Poisson Regression (`PoissonRegressor`) with `Exposure` as weights

We use **MSE** (Mean Squared Error) for initial comparison, then **Poisson deviance**, more suitable for count data.

### 5.1 Linear Regression (Baseline)

In [10]:
lin_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('linreg', LinearRegression())
])

lin_pipeline.fit(X_train, y_train)
y_pred_lin = lin_pipeline.predict(X_test)

mse_lin = mean_squared_error(y_test, y_pred_lin)
r2_lin = r2_score(y_test, y_pred_lin)

print(f"[Linear Regression] MSE = {mse_lin:.6f}")
print(f"[Linear Regression] R²  = {r2_lin:.6f}")

[Linear Regression] MSE = 0.057193
[Linear Regression] R²  = 0.012396


### 5.2 Decision Tree

In [11]:
tree = DecisionTreeRegressor(max_depth=10, random_state=42)

tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f"[Decision Tree] MSE = {mse_tree:.6f}")

[Decision Tree] MSE = 0.057129


### 5.3 Random Forest

In [12]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"[Random Forest] MSE = {mse_rf:.6f}")

[Random Forest] MSE = 0.056007


### 5.4 Gradient Boosting

In [13]:
gb = GradientBoostingRegressor(random_state=42)

gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
print(f"[Gradient Boosting] MSE = {mse_gb:.6f}")

[Gradient Boosting] MSE = 0.056125


### 5.5 Linear SVR

In [14]:
svr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', LinearSVR(random_state=42, dual='auto'))
])

svr_pipeline.fit(X_train, y_train)
y_pred_svr = svr_pipeline.predict(X_test)

mse_svr = mean_squared_error(y_test, y_pred_svr)
print(f"[SVR] MSE = {mse_svr:.6f}")

[SVR] MSE = 0.060792


### 5.6 Poisson Regression (Poisson GLM)

In [15]:
# Prepare data for Poisson (remove Exposure from features)
X_train_pois = X_train.drop(columns=["Exposure"])
X_test_pois = X_test.drop(columns=["Exposure"])

pois = PoissonRegressor(alpha=1e-4, max_iter=1000)

# Use exposure as weights (approximation of offset)
pois.fit(X_train_pois, y_train, sample_weight=exp_train)
y_pred_pois = pois.predict(X_test_pois)

mse_pois = mean_squared_error(y_test, y_pred_pois)
print(f"[Poisson Regression] MSE = {mse_pois:.6f}")

[Poisson Regression] MSE = 0.057835


## 6. Model Comparison

We now compare models using two metrics:

- **MSE**: Mean Squared Error (classic, but not ideal for Poisson data)
- **Poisson deviance**: Metric suitable for count data (lower = better model)

In [16]:
def safe_pred(y_pred):
    # Poisson deviance requires strictly positive predictions
    return np.maximum(y_pred, 1e-9)

results = pd.DataFrame({
    'Model': [
        'Linear Regression',
        'Decision Tree',
        'Random Forest',
        'Gradient Boosting',
        'Linear SVR',
        'Poisson Regression'
    ],
    'MSE': [
        mse_lin,
        mse_tree,
        mse_rf,
        mse_gb,
        mse_svr,
        mse_pois
    ],
    'Poisson Deviance': [
        mean_poisson_deviance(y_test, safe_pred(y_pred_lin)),
        mean_poisson_deviance(y_test, safe_pred(y_pred_tree)),
        mean_poisson_deviance(y_test, safe_pred(y_pred_rf)),
        mean_poisson_deviance(y_test, safe_pred(y_pred_gb)),
        mean_poisson_deviance(y_test, safe_pred(y_pred_svr)),
        mean_poisson_deviance(y_test, safe_pred(y_pred_pois)),
    ]
})

results.sort_values(by='Poisson Deviance')

,Model,MSE,Poisson Deviance
2,Random Forest,0.056007,0.296271
3,Gradient Boosting,0.056125,0.298513
1,Decision Tree,0.057129,0.305238
0,Linear Regression,0.057193,0.315877
5,Poisson Regression,0.057835,0.319758
4,Linear SVR,0.060792,2.125633


In [17]:
# Add R² for complete comparison
results['R2'] = [
    r2_score(y_test, y_pred_lin),
    r2_score(y_test, y_pred_tree),
    r2_score(y_test, y_pred_rf),
    r2_score(y_test, y_pred_gb),
    r2_score(y_test, y_pred_svr),
    r2_score(y_test, y_pred_pois)
]

results.sort_values(by='R2', ascending=False)

,Model,MSE,Poisson Deviance,R2
2,Random Forest,0.056007,0.296271,0.032902
3,Gradient Boosting,0.056125,0.298513,0.030842
1,Decision Tree,0.057129,0.305238,0.013516
0,Linear Regression,0.057193,0.315877,0.012396
5,Poisson Regression,0.057835,0.319758,0.001311
4,Linear SVR,0.060792,2.125633,-0.049741


**Interpretation:**

- **Random Forest** and **Gradient Boosting** achieve the best performance in terms of both MSE and Poisson deviance
- **Poisson Regression**, despite lower MSE than Linear SVR, shows intermediate Poisson deviance — theoretically appropriate for count data but limited by its log-linear structure
- **Linear SVR** has very high Poisson deviance (>2), suggesting it predicts negative or near-zero values, which is problematic for count data
- **R² is low for all models** (~3% max), which is expected for rare count data where individual predictions are inherently uncertain

## 7. Hyperparameter Optimization (Poisson GLM)

We use **GridSearchCV** to find the optimal regularization parameter for Poisson regression.

In [18]:
# Parameter grid for Poisson
param_grid_pois = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1.0]
}

In [19]:
# Base Poisson model (GLM log-linear structure)
pois_base = PoissonRegressor(
    max_iter=2000,
    fit_intercept=True
)

# GridSearchCV:
# - tries all hyperparameter combinations in param_grid_pois
# - for each combination, performs cross-validation (cv=3)
# - evaluates models with neg_mean_poisson_deviance
grid_pois = GridSearchCV(
    estimator=pois_base,
    param_grid=param_grid_pois,
    scoring="neg_mean_poisson_deviance",
    cv=3,
    n_jobs=-1,
    verbose=2
)

# Pass exposure as weights
grid_pois.fit(X_train_pois, y_train, sample_weight=exp_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=PoissonRegressor(max_iter=2000), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]},
             scoring='neg_mean_poisson_deviance', verbose=2)

In [20]:
print("Best alpha found:", grid_pois.best_params_["alpha"])
print("Best mean deviance (CV):", -grid_pois.best_score_)

Best alpha found: 0.0001
Best mean deviance (CV): 0.3185226923615155


In [21]:
# Best Poisson model found by GridSearchCV
pois_best = grid_pois.best_estimator_

# Predictions on train and test
y_pred_pois_train = pois_best.predict(X_train_pois)
y_pred_pois_test = pois_best.predict(X_test_pois)

# MSE and Poisson deviance on test
mse_pois_test = mean_squared_error(y_test, y_pred_pois_test)
dev_pois_test = mean_poisson_deviance(y_test, y_pred_pois_test)

print(f"Optimized Poisson MSE (test)      : {mse_pois_test:.6f}")
print(f"Optimized Poisson deviance (test) : {dev_pois_test:.6f}")

Optimized Poisson MSE (test)      : 0.057835
Optimized Poisson deviance (test) : 0.319758


**Interpretation:**

The Poisson regression optimization via GridSearchCV confirmed that the initial model was already using the optimal hyperparameter (alpha = 1e-4).
The performance obtained after optimization (MSE = 0.057835, deviance = 0.319758) is identical to the base Poisson model.
This result is consistent with the nature of the Poisson GLM model, which remains limited by its log-linear structure.
To achieve significant gains, it would be necessary to introduce non-linear transformations (splines, interactions) or more flexible Poisson models (GAM, XGBoost Poisson).

## 8. Effect of Resampling on Models

### 8.1 Building a Balanced Dataset

In [22]:
df_resampled = df1.copy()

# Isolate claims (ClaimNb > 0)
df_pos = df1[df1["ClaimNb"] > 0]

# Duplicate claims 3x
df_resampled = pd.concat([
    df1,
    df_pos,
    df_pos,
    df_pos
], ignore_index=True)

print("Original mean:", df1["ClaimNb"].mean())
print("Resampled mean:", df_resampled["ClaimNb"].mean())

Original mean: 0.05329171203299764
Resampled mean: 0.1852281654848035


### 8.2 Train/Test Split on Resampled Data

In [23]:
X_res = df_resampled.drop(columns=["ClaimNb", "Frequency", "IDpol"])
y_res = df_resampled["ClaimNb"]
expo_res = df_resampled["Exposure"]

X_train_res, X_test_res, y_train_res, y_test_res, expo_train_res, expo_test_res = train_test_split(
    X_res, y_res, expo_res, test_size=0.2, random_state=42
)

### 8.3 Poisson Regression on Resampled Data

In [24]:
# Prepare X for Poisson: remove Exposure
X_train_pois_res = X_train_res.drop(columns=["Exposure"])
X_test_pois_res = X_test_res.drop(columns=["Exposure"])

# Fit
pois_res = PoissonRegressor(alpha=1e-4, max_iter=1000)
pois_res.fit(X_train_pois_res, y_train_res, sample_weight=expo_train_res)

# Predict
y_pred_pois_res = pois_res.predict(X_test_pois_res)

# Safe predictions for deviance
y_pred_safe_res = np.maximum(y_pred_pois_res, 1e-9)

print("Resampled Poisson MSE:", mean_squared_error(y_test_res, y_pred_pois_res))
print("Resampled Poisson deviance:", mean_poisson_deviance(y_test_res, y_pred_safe_res))

Resampled Poisson MSE: 0.17719903583090857
Resampled Poisson deviance: 0.6601077743837559


## 9. Conclusion

### Key Findings

| Model | MSE | Poisson Deviance | R² |
|-------|-----|------------------|----|
| **Random Forest** | 0.056007 | 0.296271 | 0.0329 |
| **Gradient Boosting** | 0.056125 | 0.298513 | 0.0308 |
| Decision Tree | 0.057129 | 0.305238 | 0.0135 |
| Linear Regression | 0.057193 | 0.315877 | 0.0124 |
| Poisson Regression | 0.057835 | 0.319758 | 0.0013 |
| Linear SVR | 0.060792 | 2.125633 | -0.0497 |

### Recommendations

1. **For Predictive Accuracy**: Use **Random Forest** or **Gradient Boosting** — they achieve the best MSE and Poisson deviance

2. **For Regulatory Compliance**: Use **Poisson GLM** — it offers interpretable coefficients required by insurance regulators

3. **For Production**: Consider **XGBoost with Poisson objective** — combines ensemble performance with proper count data handling

### Future Improvements

- Implement GAM (Generalized Additive Models) for non-linear effects with interpretability
- Try Zero-Inflated Poisson for excess zeros
- Add feature interactions (Age × Vehicle type)
- Use SHAP values for model explanation